<a href="https://colab.research.google.com/github/Jyoshitha04/SQL/blob/main/Netflix_SQL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Netflix Dataset Analysis**

In [ ]:
import pandas as pd
import sqlite3 as sql

# Step 1: Load CSV into pandas DataFrame
df = pd.read_csv('netflix_titles.csv')

# Step 2: Connect to SQLite database (creates the database if it doesn't exist)
conn = sql.connect('database_netflix.db')

try:
    # Step 3: Write the DataFrame to the SQLite database
    df.to_sql('netflix_titles', conn, if_exists='replace', index=False)

    # Verify: Query the database
    result = pd.read_sql('SELECT * FROM netflix_titles', conn)
    print(result)

except Exception as e:
    print(f"An error occurred: {e}")


     show_id     type                  title         director  \
0         s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1         s2  TV Show          Blood & Water             None   
2         s3  TV Show              Ganglands  Julien Leclercq   
3         s4  TV Show  Jailbirds New Orleans             None   
4         s5  TV Show           Kota Factory             None   
...      ...      ...                    ...              ...   
8802   s8803    Movie                 Zodiac    David Fincher   
8803   s8804  TV Show            Zombie Dumb             None   
8804   s8805    Movie             Zombieland  Ruben Fleischer   
8805   s8806    Movie                   Zoom     Peter Hewitt   
8806   s8807    Movie                 Zubaan      Mozez Singh   

                                                   cast        country  \
0                                                  None  United States   
1     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa 

# **Analysis Queries**

# 1. Count the Number of Movies vs TV Shows

In [ ]:
query_1 = '''
    SELECT
        type,
        COUNT(*)
    FROM netflix_titles
    GROUP BY type;
    '''
result = pd.read_sql(query_1, conn)
print(result)

      type  COUNT(*)
0    Movie      6131
1  TV Show      2676


# Find the Most Common Rating for Movies and TV Shows

In [ ]:
query_2 = '''
    WITH RatingCounts AS (
        SELECT
            type,
            rating,
            COUNT(*) AS rating_count
        FROM netflix_titles
        GROUP BY type, rating
    ),
    RankedRatings AS (
        SELECT
            type,
            rating,
            rating_count,
            RANK() OVER (PARTITION BY type ORDER BY rating_count DESC) AS rank
        FROM RatingCounts
    )
    SELECT
        type,
        rating AS most_frequent_rating
    FROM RankedRatings
    WHERE rank = 1;
'''

# Execute the second query and print the results
result_2 = pd.read_sql(query_2, conn)
print(result_2)


      type most_frequent_rating
0    Movie                TV-MA
1  TV Show                TV-MA


# 3. List All Movies Released in a Specific Year (e.g., 2020)

In [ ]:
query_3 = '''
    SELECT *
    FROM netflix_titles
    WHERE release_year = 2020;
'''
result = pd.read_sql(query_3, conn)
print(result)

    show_id     type                                              title  \
0        s1    Movie                               Dick Johnson Is Dead   
1       s17    Movie  Europe's Most Dangerous Man: Otto Skorzeny in ...   
2       s18  TV Show                                    Falsa identidad   
3       s33  TV Show                                      Sex Education   
4       s35  TV Show                            Tayo and Little Wizards   
..      ...      ...                                                ...   
948   s5973    Movie                                   #cats_the_mewvie   
949   s7595    Movie                 Norm of the North: Family Vacation   
950   s8100    Movie                                        Straight Up   
951   s8126  TV Show                                        Super Wings   
952   s8133  TV Show          Surviving R. Kelly Part II: The Reckoning   

                                          director  \
0                                  Kirsten Jo

# 4. Find the top 5 countries with the most content on Netflix

In [ ]:
# First, load the data into pandas from the database
df = pd.read_sql('SELECT country FROM netflix_titles WHERE country IS NOT NULL', conn)

# Split the 'country' column by commas into multiple rows
df['country'] = df['country'].str.split(',')

# Explode the list of countries into separate rows
df_exploded = df.explode('country')

# Trim any leading/trailing whitespace from country names
df_exploded['country'] = df_exploded['country'].str.strip()

# Group by country and count the occurrences
country_counts = df_exploded.groupby('country').size().reset_index(name='total_content')

# Sort by total_content in descending order and select the top 5
top_5_countries = country_counts.sort_values(by='total_content', ascending=False).head(5)

print(top_5_countries)


            country  total_content
116   United States           3690
46            India           1046
115  United Kingdom            806
21           Canada            445
37           France            393


# 5. Identify the longest movie

In [ ]:
query_5 = '''
    SELECT
        *
    FROM netflix_titles
    WHERE type = 'Movie'
    ORDER BY CAST(substr(duration, 1, instr(duration, ' ') - 1) AS INTEGER) DESC
'''

# Execute the query and display the result
result = pd.read_sql(query_5, conn)
print(result)

     show_id   type                                 title  \
0      s4254  Movie            Black Mirror: Bandersnatch   
1       s718  Movie           Headspace: Unwind Your Mind   
2      s2492  Movie                The School of Mischief   
3      s2488  Movie                        No Longer kids   
4      s2485  Movie                    Lock Your Girls In   
...      ...    ...                                   ...   
6126   s2714  Movie                           Sol Levante   
6127   s3778  Movie                                Silent   
6128   s5542  Movie                       Louis C.K. 2017   
6129   s5795  Movie                 Louis C.K.: Hilarious   
6130   s5814  Movie  Louis C.K.: Live at the Comedy Store   

                               director  \
0                                  None   
1                                  None   
2                Houssam El-Din Mustafa   
3                       Samir Al Asfory   
4                     Fouad El-Mohandes   
...      

# 6. Find content added in the last 5 years

In [ ]:
query_last_5_years = '''
    SELECT *
    FROM netflix_titles
    WHERE date_added >= DATE('now', '-5 years')
'''
# Execute the query and fetch the results
recent_content = pd.read_sql(query_last_5_years, conn)

# Display the results
print(recent_content)

     show_id     type                  title                 director  \
0         s1    Movie   Dick Johnson Is Dead          Kirsten Johnson   
1         s2  TV Show          Blood & Water                     None   
2         s3  TV Show              Ganglands          Julien Leclercq   
3         s4  TV Show  Jailbirds New Orleans                     None   
4         s5  TV Show           Kota Factory                     None   
...      ...      ...                    ...                      ...   
3454   s8786  TV Show                    YOM                     None   
3455   s8787    Movie           You Can Tutu              James Brown   
3456   s8790    Movie    You Changed My Life      Cathy Garcia-Molina   
3457   s8799    Movie               Zed Plus  Chandra Prakash Dwivedi   
3458   s8802    Movie                Zinzana          Majid Al Ansari   

                                                   cast  \
0                                                  None   
1    

# 7. Find all the movies/TV shows by director 'Rajiv Chilaka'!

In [ ]:
# Step 1: Load the data from the database into a pandas DataFrame
df = pd.read_sql('SELECT * FROM netflix_titles', conn)

# Step 2: Split the 'director' column by commas into lists
df['director'] = df['director'].str.split(',')

# Step 3: Explode the lists into separate rows
df_exploded = df.explode('director')

# Step 4: Trim any leading or trailing spaces
df_exploded['director'] = df_exploded['director'].str.strip()

# Step 5: Filter for the specific director name 'Rajiv Chilaka'
filtered_df = df_exploded[df_exploded['director'] == 'Rajiv Chilaka']

# Step 6: Display the result
print(filtered_df)


     show_id   type                                              title  \
406     s407  Movie                       Chhota Bheem - Neeli Pahaadi   
407     s408  Movie                              Chhota Bheem & Ganesh   
408     s409  Movie                 Chhota Bheem & Krishna: Mayanagari   
409     s410  Movie  Chhota Bheem & Krishna: Pataliputra- City of t...   
410     s411  Movie                 Chhota Bheem And The Broken Amulet   
411     s412  Movie             Chhota Bheem And The Crown of Valhalla   
412     s413  Movie               Chhota Bheem and the Incan Adventure   
413     s414  Movie                Chhota Bheem and The ShiNobi Secret   
414     s415  Movie                           Chhota Bheem Aur Hanuman   
415     s416  Movie                           Chhota Bheem aur Krishna   
416     s417  Movie                Chhota Bheem aur Krishna vs Zimbara   
419     s420  Movie                      Chhota Bheem: Bheem vs Aliens   
420     s421  Movie               Chho

# 8. List all TV shows with more than 5 seasons


In [ ]:
query_8 = '''
    SELECT *
    FROM netflix_titles
    WHERE
        type = 'TV Show'
        AND CAST(SUBSTR(duration, 1, INSTR(duration, ' ') - 1) AS INTEGER) > 5
'''

# Execute the query and display the result
tv_shows = pd.read_sql(query_8, conn)
print(tv_shows)

   show_id     type                          title         director  \
0       s9  TV Show  The Great British Baking Show  Andy Devonshire   
1      s56  TV Show                      Nailed It             None   
2      s66  TV Show                   Numberblocks             None   
3      s68  TV Show              Saved by the Bell             None   
4      s83  TV Show                        Lucifer             None   
..     ...      ...                            ...              ...   
94   s8190  TV Show         The Andy Griffith Show             None   
95   s8379  TV Show                     The L Word             None   
96   s8443  TV Show              The Office (U.S.)             None   
97   s8558  TV Show                  The West Wing             None   
98   s8711  TV Show                          Weeds             None   

                                                 cast                country  \
0   Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...         Unite

# 9. Count the number of content items in each genre


In [ ]:
query_9 = '''
WITH RECURSIVE split_genres AS (
    SELECT
        TRIM(SUBSTR(listed_in, 1, INSTR(listed_in || ',', ',') - 1)) AS genre,
        SUBSTR(listed_in, INSTR(listed_in || ',', ',') + 1) AS remaining
    FROM netflix_titles

    UNION ALL

    SELECT
        TRIM(SUBSTR(remaining, 1, INSTR(remaining || ',', ',') - 1)),
        SUBSTR(remaining, INSTR(remaining || ',', ',') + 1)
    FROM split_genres
    WHERE remaining <> ''
)

SELECT genre, COUNT(*) AS total_content
FROM split_genres
GROUP BY genre
'''

# Execute the query and display the result
genre_counts = pd.read_sql(query_9, conn)
print(genre_counts)


                           genre  total_content
0             Action & Adventure            859
1                 Anime Features             71
2                   Anime Series            176
3               British TV Shows            253
4       Children & Family Movies            641
5              Classic & Cult TV             28
6                 Classic Movies            116
7                       Comedies           1674
8                 Crime TV Shows            470
9                    Cult Movies             71
10                 Documentaries            869
11                    Docuseries            395
12                        Dramas           2427
13          Faith & Spirituality             65
14                 Horror Movies            357
15            Independent Movies            756
16          International Movies           2752
17        International TV Shows           1351
18                      Kids' TV            451
19               Korean TV Shows        

# 10. Find each year and the average numbers of content release by India on netflix. Return top 5 year with highest avg content release !


In [ ]:
query_10 = '''
SELECT
    country,
    release_year,
    COUNT(show_id) AS total_release,
    ROUND(
        COUNT(show_id) * 1.0 /
        (SELECT COUNT(show_id) FROM netflix_titles WHERE country = 'India') * 100,
        2
    ) AS avg_release
FROM netflix_titles
WHERE country = 'India'
GROUP BY country, release_year
ORDER BY avg_release DESC
LIMIT 5
'''

# Execute the query and display the result
result_10 = pd.read_sql(query_10, conn)
print(result_10)


  country  release_year  total_release  avg_release
0   India          2017            101        10.39
1   India          2018             94         9.67
2   India          2019             87         8.95
3   India          2020             75         7.72
4   India          2016             73         7.51


# 11. List all movies that are documentaries

In [ ]:
query_11 = '''
    SELECT * FROM netflix_titles
    WHERE listed_in LIKE '%Documentaries'
'''

result_11 = pd.read_sql(query_11, conn)
print(result_11)

    show_id   type                                        title  \
0        s1  Movie                         Dick Johnson Is Dead   
1       s46  Movie                       My Heroes Were Cowboys   
2      s118  Movie                                Final Account   
3      s222  Movie  Bob Ross: Happy Accidents, Betrayal & Greed   
4      s305  Movie          Lady Boss: The Jackie Collins Story   
..      ...    ...                                          ...   
378   s8724  Movie                     What Makes a Psychopath?   
379   s8738  Movie                   Why Are We Getting So Fat?   
380   s8739  Movie                                     Why Knot   
381   s8740  Movie           Why We Fight: The Battle of Russia   
382   s8764  Movie              WWII: Report from the Aleutians   

                                   director            cast  \
0                           Kirsten Johnson            None   
1                               Tyler Greco            None   
2     

# 12. Find all content without a director

In [ ]:
query_12 = '''
    SELECT * FROM netflix_titles
WHERE director IS NULL

'''

result_12 = pd.read_sql(query_12, conn)
print(result_12)

     show_id     type                                title director  \
0         s2  TV Show                        Blood & Water     None   
1         s4  TV Show                Jailbirds New Orleans     None   
2         s5  TV Show                         Kota Factory     None   
3        s11  TV Show  Vendetta: Truth, Lies and The Mafia     None   
4        s15  TV Show      Crime Stories: India Detectives     None   
...      ...      ...                                  ...      ...   
2629   s8796  TV Show                      Yu-Gi-Oh! Arc-V     None   
2630   s8797  TV Show                           Yunus Emre     None   
2631   s8798  TV Show                            Zak Storm     None   
2632   s8801  TV Show                   Zindagi Gulzar Hai     None   
2633   s8804  TV Show                          Zombie Dumb     None   

                                                   cast  \
0     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   
1                            

# 13. Find how many movies actor 'Salman Khan' appeared in last 10 years!


In [ ]:
query_13 = '''
SELECT *
FROM netflix_titles
WHERE
    "cast" LIKE '%Salman Khan%'
    AND release_year > strftime('%Y', 'now', '-10 years')
'''

# Execute the query and display the result
result_13 = pd.read_sql(query_13, conn)
print(result_13)


  show_id   type                 title             director  \
0   s2340  Movie  Prem Ratan Dhan Payo   Sooraj R. Barjatya   
1   s2757  Movie             Paharganj  Rakesh Ranjan Kumar   

                                                cast country date_added  \
0  Salman Khan, Sonam Kapoor, Anupam Kher, Neil N...   India  25-Jun-20   
1  Lorena Franco, Bijesh Jayarajan, Neet Chowdhar...   India  31-Mar-20   

   release_year rating duration  \
0          2015  TV-14  164 min   
1          2019  TV-MA  107 min   

                                          listed_in  \
0  Action & Adventure, Dramas, International Movies   
1  Dramas, Independent Movies, International Movies   

                                         description  
0  While recovering from an assassination attempt...  
1  In the underbelly of a Delhi neighborhood, a w...  


# 14. Find the top 10 actors who have appeared in the highest number of movies produced in India.


In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame with the Netflix dataset
# Filter for Indian movies
india_movies = df[df['country'] == 'India']

# Split the cast column and explode to get individual actors
top_actors = india_movies['cast'].str.split(', ').explode()

# Count the occurrences of each actor
top_actors_count = top_actors.value_counts().head(10)

# Display the results
print(top_actors_count)


cast
Anupam Kher         40
Shah Rukh Khan      32
Naseeruddin Shah    29
Amitabh Bachchan    28
Akshay Kumar        28
Paresh Rawal        27
Om Puri             26
Kareena Kapoor      24
Boman Irani         23
Ajay Devgn          20
Name: count, dtype: int64


# Question 15:
Categorize the content based on the presence of the keywords 'kill' and 'violence' in
the description field. Label content containing these keywords as 'Bad' and all other
content as 'Good'. Count how many items fall into each category.

In [ ]:
query_15 = '''
    SELECT
        category,
        type,
        COUNT(*) AS content_count
    FROM (
        SELECT
            *,
            CASE
                WHEN description LIKE '%kill%' OR description LIKE '%violence%' THEN 'Bad'
                ELSE 'Good'
            END AS category
        FROM netflix_titles
    ) AS categorized_content
    GROUP BY category, type
    ORDER BY type;
'''
# Execute the query and fetch the results
content_categories = pd.read_sql(query_15, conn)

# Display the results
print(content_categories)


  category     type  content_count
0      Bad    Movie            251
1     Good    Movie           5880
2      Bad  TV Show             91
3     Good  TV Show           2585
